In [0]:
try: 
  dbutils.notebook.run(path = "/Users/siddardha.kaja@aircanada.ca/DP-ETL Framework/prod/Generic-Mount", arguments={'project_name': 'pm_ap_conn'}, timeout_seconds=28000)
  print("pm_ap_conn is mounted")
except:
  print("pm_ap_conn is already mounted")

pm_ap_conn is already mounted


In [0]:
pip install oauth2client

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
pip install google-api-python-client

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
def fetchYoutubeComments(**kwargs):
  """
  The following params can be passed to the function to fetch the youtube video comments by searching them and the comments and their details are saved in a delta table:

  keyword (required) : search keyword can pass multiple keywords by seperating them with `|`
  apiVersion(required) : Its `v3` at the time of development
  pathToServiceAccountJsonFile(required) : contact your manager for this
  impersonationEmail(required) : contact your manager for this
  locationDetails(optional) : pass lattitude, longitude and search radius in a list. Radius is in miles. Eg.,[lattitude, longitude, radius]
  noResultsToFetch(optional) : depending on api request limit, pass the number else the default is 50. Should be in the mutiples of 50
  searchOrder(optional) : The order parameter specifies the method that will be used to order resources in the API response. The default value is relevance. search order can 'date', 'relevance', 'rating', 'title', 'videoCount', 'viewCount'. 
  regionCode(optional) : The regionCode parameter instructs the API to return search results for videos that can be viewed in the specified country
  languageCode(optional) : The relevanceLanguage parameter instructs the API to return search results that are most relevant to the specified language.
  videoId(optional) : Declaring this param forces the function to fetch comments for this specific video
  startDate(optional) : this indicates that the API response should only contain resources created at or after the specified date.
  endDate(optional) : this indicates that the API response should only contain resources created before or at the specified date

  usage:
  fetchYoutubeComments(
  keyword = "air canada",
  apiVersion = 'v3',
  noResultsToFetch = 100,
  searchOrder = 'viewCount',
  regionCode = 'CA',
  startDate = '2023-12-01',
  endDate = '2024-01-01',
  impersonationEmail = 'XXXXXX-001@CCCCCC315001.iam.gserviceaccount.com',
  pathToServiceAccountJsonFile = '/dbfs/mnt/Sandbox/XXX/XXX/XXXXXXXXXX.json'
  )

  The data will be deposited in the predetermined tables
  """
  from typing import Optional, Tuple
  import datetime
  from pyspark.sql.functions import to_date, col
  from pyspark.sql import SparkSession
  spark = SparkSession.builder.getOrCreate()
  spark.conf.set("spark.sql.shuffle.partitions", "31")
  spark.sql('SET spark.databricks.delta.retentionDurationCheck.enabled=false')
  channel = 'youtube'
  for key, value in kwargs.items():
    globals()[key] = value


  def setConnection(apiVersion: str, channel: str, impersonationEmail: str, pathToServiceAccountJsonFile: str):
    #creates service to youtube
    from googleapiclient import discovery
    from oauth2client.service_account import ServiceAccountCredentials
    import httplib2

    OAuthScopes = 'https://www.googleapis.com/auth/youtube.force-ssl'
    impersonationEmail = impersonationEmail
    pathToServiceAccountJsonFile = pathToServiceAccountJsonFile
    credentials = ServiceAccountCredentials.from_json_keyfile_name(pathToServiceAccountJsonFile,scopes=OAuthScopes)
    if impersonationEmail:
      credentials = credentials.create_delegated(impersonationEmail)
    http = credentials.authorize(httplib2.Http())
    service = discovery.build(channel, apiVersion, http=http)
    return service
  
  def fetchcurrts(timezone, format):
    #get current datetime from the mentioned timezone in the required format
    import pytz
    from datetime import datetime

    est = pytz.timezone(timezone) 

    dt = datetime.now(est).strftime(format)
    return dt

  def takeFewVideos(li, liSize): 
    ## splits a list into multiple smaller size lists. we use to control the number of videoids for a request
    for i in range(0, len(li), liSize):  
        yield li[i:i + liSize] 
  
  def getVideoIds(service, keyword, locationDetails, maxNoResults, searchOrder, regionCode, relevanceLanguage, publishBeginDate, publishEndDate):
    #fetches the vidoeIds based on the search params

    if locationDetails is not None:
      createLocation = str(locationDetails[0])+','+str(locationDetails[1])
      createRadius = str(locationDetails[2])+'mi'                                               
    else:
      createLocation = None    
      createRadius =  None    

    if publishBeginDate is not None:
      publishBeginDate = publishBeginDate+'T00:00:00Z'

    if publishEndDate is not None:
      publishEndDate = publishEndDate+'T00:00:00Z'

    request = service.search().list(
          part="snippet",
          location=createLocation,
          locationRadius=createRadius,
          maxResults=maxNoResults,
          order=searchOrder,
          q=keyword,
          type = 'video',
          regionCode = regionCode,
          relevanceLanguage = relevanceLanguage,     
          publishedAfter=publishBeginDate,
          publishedBefore=publishEndDate                                                                                      
      )
    response = request.execute()
    nextPageToken = response['nextPageToken']
    regionCode = response['regionCode']
    fullList = []
    for i in response['items']:
      tempList = [i['id']['videoId'], spark.sql("select cast(from_utc_timestamp(current_timestamp, 'America/Toronto') as string)").collect()[0][0],  regionCode, 'mainpage']
      fullList.append(tempList)
    return fullList, nextPageToken
  
  def getVideoIdsNextPage(service, fullList, nextPageToken, locationDetails, maxNoResults, searchOrder, regionCode, relevanceLanguage, publishBeginDate, publishEndDate):
    #get vidoeIds for the next page as each page can hold only 50 items
    if locationDetails is not None:
      createLocation = str(locationDetails[0])+','+str(locationDetails[1])

      createRadius = str(locationDetails[2])+'mi' 
                                                  
    else:
      createLocation = None    
      createRadius =  None
    if publishBeginDate is not None:
      publishBeginDate = publishBeginDate+'T00:00:00Z'

    if publishEndDate is not None:
      publishEndDate = publishEndDate+'T00:00:00Z'

    request = service.search().list(
          part="snippet",
          location=createLocation,
          locationRadius=createRadius,
          maxResults=maxNoResults,
          order=searchOrder,
          pageToken=nextPageToken,
          type = 'video',
          regionCode = regionCode,
          relevanceLanguage = relevanceLanguage,
          publishedAfter=publishBeginDate,
          publishedBefore=publishEndDate                                                                                    
      )
    response = request.execute()
    
    if 'nextPageToken' in response:
      nextPageToken = response['nextPageToken']
      regionCode = response['regionCode']
      for i in response['items']:
        tempList = [i['id']['videoId'], spark.sql("select cast(from_utc_timestamp(current_timestamp, 'America/Toronto') as string)").collect()[0][0],  regionCode, nextPageToken]
        fullList.append(tempList)
    else:
      nextPageToken = None
    
    return fullList, nextPageToken
  
  def getComments(service, videoId, commentSearchTerm):
    # fetches the comments of a video
    from pyspark.sql import SparkSession
    from pyspark.sql.types import StructType, StructField, StringType
    from pyspark.sql.functions import lit
    request = service.commentThreads().list(
          part="snippet,replies",
          searchTerms=commentSearchTerm,
          videoId=videoId,
          maxResults = 100,
          textFormat = 'plainText'
      )
    try:
      response = request.execute()
      commentsList = extractComments(response, videoId)
    except:
      response = [None]
      commentsList = [[str(videoId), None, None, None, None, None, None, None, None, None, None, None, None, None, None]]
  
    if 'nextPageToken' in response:
      nextPageToken = response['nextPageToken']
      while nextPageToken != None:
        res = getCommentsNextpage(service = service, commentSearchTerm = commentSearchTerm, videoId = videoId, nextPageToken = nextPageToken, commentsList = commentsList)
        commentsList = res[0]
        nextPageToken = res[1]
    commentSchema = StructType([
      StructField("videoId", StringType(), False),
      StructField("commentParentId", StringType(), True),
      StructField("commentKind", StringType(), True),
      StructField("commentEtag", StringType(), True),
      StructField("commentId", StringType(), True),
      StructField("comment", StringType(), True),
      StructField("commentAuthor", StringType(), True),
      StructField("commentAuthorChannelId", StringType(), True),
      StructField("commentCanRate", StringType(), True),
      StructField("commentViewerRating", StringType(), True),
      StructField("commentLikeCount", StringType(), True),
      StructField("commentReplyCount", StringType(), True),
      StructField("commentPublishedAt", StringType(), True),
      StructField("commentUpdatedAt", StringType(), True),
      StructField("commentModerationStatus", StringType(), True)
      ])
    sparkDfComments = createSparkDf(commentsList, commentSchema)
    sparkDfComments = sparkDfComments.withColumn('commentsFetchTimestampEst', lit( fetchcurrts('Canada/Eastern','%Y%m%d%H%M%S'))).withColumn('commentsUpdateTimestampEst', lit( fetchcurrts('Canada/Eastern','%Y%m%d%H%M%S')))

    return sparkDfComments

  def getCommentsNextpage(service, commentSearchTerm, videoId, nextPageToken, commentsList):
    # each page can hold only 50 items, if there are more comments, this function fetches

    request = service.commentThreads().list(
        part="snippet,replies",
        searchTerms=commentSearchTerm,
        videoId=videoId,
        maxResults = 100,
        textFormat = 'plainText',
        pageToken = nextPageToken
    )
    try:
      response = request.execute()
      commentsListNextPage = extractComments(response, videoId)
    except:
      response = [None]
      commentsListNextPage = [[str(videoId), None, None, None, None, None, None, None, None, None, None, None, None, None, None]]
    commentsList.extend(commentsListNextPage)

    if 'nextPageToken' in response:
      nextPageToken = response['nextPageToken']
    else:
      nextPageToken = None
    
    return commentsList, nextPageToken

  def createSparkDf(data, schema):
    # creates pyspark df
    from pyspark.sql import SparkSession
    from pyspark.sql.types import StructType, StructField, StringType
  
    df = spark.createDataFrame(data, schema)

    return df
  
  def getVideoDetails(service, videoListStr):
    # fetches video attributes
    from pyspark.sql import SparkSession
    from pyspark.sql.types import StructType, StructField, StringType
    request = service.videos().list(
        part="snippet,contentDetails,statistics",
        id=videoListStr
    )
    videoDetails = request.execute()
    videoDetailsList = []
    pos = 0
    for video in videoDetails['items']:
      currentVideoList = []

      try:
        videoId = video['id']
      except KeyError:
        videoId = None
      currentVideoList.insert(0, videoId)

      try:
        kind = video['kind']
      except KeyError:
        kind = None
      currentVideoList.insert(1, kind)

      try:
        pubishedAt = video['snippet']['publishedAt']
      except KeyError:
        pubishedAt = None
      currentVideoList.insert(2, pubishedAt)

      try:
        channelId = video['snippet']['channelId']
      except KeyError:
        channelId = None
      currentVideoList.insert(3, channelId)

      try:
        channelTitle = video['snippet']['channelTitle']
      except KeyError:
        channelTitle = None
      currentVideoList.insert(4, channelTitle)

      try:
        title = video['snippet']['title']
      except KeyError:
        title = None
      currentVideoList.insert(5, title)

      try:
        description = video['snippet']['description']
      except KeyError:
        description = None
      currentVideoList.insert(6, description)

      try:
        tags = video['snippet']['tags']
      except KeyError:
        tags = None
      currentVideoList.insert(7, tags)

      try:
        duration = video['contentDetails']['duration']
      except KeyError:
        duration = None
      currentVideoList.insert(8, duration)

      try:
        regionRestriction = video['contentDetails']['regionRestriction']
      except KeyError:
        regionRestriction = None
      currentVideoList.insert(9, regionRestriction)

      try:
        viewCount = video['statistics']['viewCount']
      except KeyError:
        viewCount = None
      currentVideoList.insert(10, viewCount)

      try:
        likeCount = video['statistics']['likeCount']
      except KeyError:
        likeCount = None
      currentVideoList.insert(11, likeCount)

      try:
        favoriteCount = video['statistics']['favoriteCount']
      except KeyError:
        favoriteCount = None
      currentVideoList.insert(12, favoriteCount)

      try:
        commentCount = video['statistics']['commentCount']
      except KeyError:
        commentCount = None
      currentVideoList.insert(13, commentCount)

      videoDetailsList.insert(pos, currentVideoList)
      pos = pos+1

    videoAttrSchema = StructType([
          StructField("videoId", StringType(), True),
          StructField("kind", StringType(), True),
          StructField("publishedAt", StringType(), True),
          StructField("channelId", StringType(), True),
          StructField("channelTitle", StringType(), True),
          StructField("title", StringType(), True),
          StructField("description", StringType(), True),
          StructField("tags", StringType(), True),
          StructField("duration", StringType(), True),
          StructField("regionRestriction", StringType(), True),
          StructField("viewCount", StringType(), True),
          StructField("likeCount", StringType(), True),
          StructField("favoriteCount", StringType(), True),
          StructField("commentCount", StringType(), True)
      ])
    sparkDfVideoAttr = createSparkDf(videoDetailsList, videoAttrSchema)
    
    return sparkDfVideoAttr
  
  def extractComments(comm, videoId):
    # extracts comments from the response object
    commentList = []
    commentPos = 0

    replyList = []
    replyPos = 0

    for comment in comm['items']:
      eachComment = []
      pos = 0

      videoId = videoId
      eachComment.insert(pos, videoId)
      pos += 1

      commentParentId = None
      eachComment.insert(pos, commentParentId)
      pos += 1

      try:
        kind = comment['snippet']['topLevelComment']['kind']
      except KeyError:
        kind = None
      eachComment.insert(pos, kind)
      pos += 1

      try:
        etag = comment['snippet']['topLevelComment']['etag']
      except Exception as e:
        etag = None
      eachComment.insert(pos, etag)
      pos += 1

      try:
        commentid = comment['snippet']['topLevelComment']['id']
      except KeyError:
        commentid = None
      eachComment.insert(pos, commentid)
      pos += 1

      try:
        textDisplay = comment['snippet']['topLevelComment']['snippet']['textDisplay']
      except KeyError:
        textDisplay = None
      eachComment.insert(pos, textDisplay)
      pos += 1

      try:
        authorDisplayName =  comment['snippet']['topLevelComment']['snippet']['authorDisplayName']
      except KeyError:
        authorDisplayName = None
      eachComment.insert(pos, authorDisplayName)
      pos += 1

      try:
        authorChannelId =  comment['snippet']['topLevelComment']['snippet']['authorChannelId']['value']
      except KeyError:
        authorChannelId = None
      eachComment.insert(pos, authorChannelId)
      pos += 1


      try:
        canRate = comment['snippet']['topLevelComment']['snippet']['canRate']
      except KeyError:
        canRate = None
      eachComment.insert(pos, canRate)
      pos += 1

      try:
        viewerRating = comment['snippet']['topLevelComment']['snippet']['viewerRating']
      except KeyError:
        viewerRating = None
      eachComment.insert(pos, viewerRating)
      pos += 1

      try:
        likeCount = comment['snippet']['topLevelComment']['snippet']['likeCount']
      except KeyError:
        likeCount = None
      eachComment.insert(pos, likeCount)
      pos += 1

      try:
        totalReplyCount = comment['snippet']['totalReplyCount']
      except KeyError:
        totalReplyCount = None

      if totalReplyCount != 0 and 'replies' in comment.keys():
        for reply in comment['replies']['comments']:
          eachReply = []
          eachReplyPos = 0

          try:
            replyvideoId = reply['snippet']['videoId']
          except KeyError:
            replyvideoId = None
          eachReply.insert(eachReplyPos, replyvideoId)
          eachReplyPos += 1

          try:
            commentParentId = reply['snippet']['parentId']
          except KeyError:
            commentParentId = None
          eachReply.insert(eachReplyPos, commentParentId)
          eachReplyPos += 1
          
          replyKind = 'youtube#reply'
          eachReply.insert(eachReplyPos, replyKind)
          eachReplyPos += 1

          try:
            replyEtag = reply['etag']
          except KeyError:
            replyEtag = None
          eachReply.insert(eachReplyPos, replyEtag)
          eachReplyPos += 1

          try:
            replyId = reply['id']
          except KeyError:
            replyId = None
          eachReply.insert(eachReplyPos, replyId)
          eachReplyPos += 1

          try:
            replyTextDisplay = reply['snippet']['textDisplay']
          except KeyError:
            replyTextDisplay = None
          eachReply.insert(eachReplyPos, replyTextDisplay)
          eachReplyPos += 1

          try:
            replyAuthorName = reply['snippet']['authorDisplayName']
          except KeyError:
            replyAuthorName = None
          eachReply.insert(eachReplyPos, replyAuthorName)
          eachReplyPos += 1

          try:
            replyChannelId = reply['snippet']['channelId']
          except KeyError:
            replyChannelId = None
          eachReply.insert(eachReplyPos, replyChannelId)
          eachReplyPos += 1

          try:
            replyCanRate = reply['snippet']['canRate']
          except KeyError:
            replyCanRate = None
          eachReply.insert(eachReplyPos, replyCanRate)
          eachReplyPos += 1

          try:
            replyViewerRating = reply['snippet']['viewerRating']
          except KeyError:
            replyViewerRating = None
          eachReply.insert(eachReplyPos, replyViewerRating)
          eachReplyPos += 1


          try:
            replyLikeCount = reply['snippet']['likeCount']
          except KeyError:
            replyLikeCount = None
          eachReply.insert(eachReplyPos, replyLikeCount)
          eachReplyPos += 1

          replyReplyCount = None
          eachReply.insert(eachReplyPos, replyReplyCount)
          eachReplyPos += 1

          try:
            replyPublishedAt = reply['snippet']['publishedAt']
          except KeyError:
            replyPublishedAt = None
          eachReply.insert(eachReplyPos, replyPublishedAt)
          eachReplyPos += 1

          try:
            replyUpdatedAt = reply['snippet']['updatedAt']
          except KeyError:
            replyUpdatedAt = None
          eachReply.insert(eachReplyPos, replyUpdatedAt)
          eachReplyPos += 1

          try:
            moderationStatus =  reply['snippet']['moderationStatus']
          except KeyError:
            moderationStatus = None
          eachReply.insert(eachReplyPos, moderationStatus)

          replyList.insert(replyPos, eachReply)
          replyPos = replyPos + 1
          
      eachComment.insert(pos, totalReplyCount)
      pos += 1

      try:
        publishedAt =  comment['snippet']['topLevelComment']['snippet']['publishedAt']
      except KeyError:
        publishedAt = None
      eachComment.insert(pos, publishedAt)
      pos += 1

      try:
        updatedAt = comment['snippet']['topLevelComment']['snippet']['updatedAt']
      except KeyError:
        updatedAt = None
      eachComment.insert(pos, updatedAt)
      pos += 1

      try:
        moderationStatus =  comment['snippet']['topLevelComment']['snippet']['moderationStatus']
      except KeyError:
        moderationStatus = None
      eachComment.insert(pos, moderationStatus)
      commentList.insert(commentPos, eachComment)
      commentPos = commentPos+1
      totalCommentsAndReplies = commentList + replyList

    return totalCommentsAndReplies
  
  def filterFurther(keyword):
    #sometimes search can return irrelevant videos, so this function filters out the unnecessary videos by making sure that either title or description or tags should contain the key work atleast once
    keywordList = keyword.split("|")
    keywordFilterString = ''
    concatString = 'or'
    for i in range(0, len(keywordList)):
      #convert to string and strip for whitespaces at beginning and ending
      keywordItem = str(keywordList[i]).strip()
      if i == (len(keywordList)-1):
        tempString = "lower(title) like '%"+keywordItem.lower()+"%' or lower(description) like '%"+keywordItem.lower()+"%' or lower(tags) like '%"+keywordItem.lower()+"%'"
        keywordFilterString = keywordFilterString+tempString
      else:
        tempString = "lower(title) like '%"+keywordItem.lower()+"%' or lower(description) like '%"+keywordItem.lower()+"%' or lower(tags) like '%"+keywordItem.lower()+"%' or "
        keywordFilterString = keywordFilterString+tempString
    return keywordFilterString

  def main(channel: str ,keyword: str,  apiVersion: str, pathToServiceAccountJsonFile: str, impersonationEmail: str, commentSearchTermsList : Optional = None, locationDetails: Optional[Tuple[str, str, int]] = None, noResultsToFetch: Optional[int] = 50, searchOrder: Optional[str] = None, regionCode: Optional[str] = None, languageCode: Optional[str] = None, videoId: Optional[str] = None, startDate: Optional[str] = None, endDate: Optional[str] = None  ):
  
    #parameters validation
    #validate keyword param

    commentsMountPath = "/mnt/Sandbox/pm_ap_conn/sid_test/youtube/daily/youtubeVideoCommentsReplies"
    videosMountPath = "/mnt/Sandbox/pm_ap_conn/sid_test/youtube/daily/youtubeVideoAttributes"
    assert isinstance(keyword, str), "keyword needs to be in string"

    
    assert isinstance(channel, str), "channel must be in string format"
    assert channel in ['youtube'], "searchOrder should be among `youtube`"
    

    #validate api_version param
    assert isinstance(apiVersion, str), "keyword needs to be in string"
    #validate location_details param
    if locationDetails:
      if isinstance(locationDetails, tuple):
        assert len(locationDetails) == 3, "locationDetails expecting 3 values longitue, lattitude and range in miles "
        assert isinstance(locationDetails[0], str ), "lattitude must be in string format"
        assert isinstance(locationDetails[1], str ), "longitude must be in string format"
        assert isinstance(locationDetails[2], int ), "Radius must be in integer format"
      else:
        raise Exception("locationDetails needs to be a tuple")

    #validate max_no_results param
    if noResultsToFetch:
      assert isinstance(noResultsToFetch, int), "noResultsToFetch must be in integer format"

    #validate search_order param
    if searchOrder:
      assert isinstance(searchOrder, str), "searchOrder must be in string format"
      assert searchOrder in ['date', 'relevance', 'rating', 'title', 'videoCount', 'viewCount'], "searchOrder should be among `date`, `rating`, `relevance`, `title`, `videoCount`, `viewCount`. Default value is `date`"

    #validate region_code param
    if regionCode: 
      assert isinstance(regionCode, str), "regionCode must be in string format"
      assert len(regionCode) == 2, "regionCode should be an ISO 3166-1 alpha-2 country code. Please refer to https://www.iso.org/obp/ui/#search"

    #validate language_code param
    if languageCode: 
      assert isinstance(languageCode, str), "languageCode must be in string format"
      assert len(languageCode) != 2, "languageCode should be an ISO 639-1 two-letter language code. Please refer to https://www.loc.gov/standards/iso639-2/php/code_list.php"
    
    assert isinstance(impersonationEmail, str), "impersonationEmail needs to be in string"

    assert isinstance(pathToServiceAccountJsonFile, str), "pathToServiceAccountJsonFile needs to be in string"

    #validate video_id param
    if videoId:
      assert isinstance(videoId, str), "videoId must be in string format"
    
    dateFormat = "%Y-%m-%d"
    if startDate:
      try:
        datetime.datetime.strptime(startDate, dateFormat)
      except ValueError:
        print("Incorrect startDate Format. It should be 'YYYY-MM-DD'")
        raise
        return
      except:
        raise
        return
    
    if endDate:
      try:
        datetime.datetime.strptime(endDate, dateFormat)
      except ValueError:
        print("Incorrect endDate Format. It should be 'YYYY-MM-DD'")
        raise
        return
      except:
        raise
        return
    


    if channel == 'youtube':
      service  = setConnection(apiVersion = apiVersion, channel = channel, impersonationEmail = impersonationEmail, pathToServiceAccountJsonFile = pathToServiceAccountJsonFile)
      

      if not videoId:
        firstPageVidoes = getVideoIds(service, keyword = keyword, locationDetails = locationDetails, maxNoResults= '50', searchOrder = searchOrder, regionCode = regionCode, relevanceLanguage = languageCode, publishBeginDate = startDate, publishEndDate = endDate)
        
        if firstPageVidoes[1]:
          nextPageToken = firstPageVidoes[1]

          fullList = firstPageVidoes[0]
          videoCount = len(firstPageVidoes[0])
          while videoCount < noResultsToFetch:
            b = getVideoIdsNextPage(service, fullList = fullList, nextPageToken = nextPageToken, locationDetails = locationDetails, maxNoResults= '50', searchOrder = searchOrder, regionCode = regionCode, relevanceLanguage = languageCode, publishBeginDate = startDate, publishEndDate = endDate)
            try:
              nextPageToken = b[1]
              fullList = b[0]
            except:
              break
            videoCount = len(b[0])

      else:
        # if the individual video id is passed, we will not fetch the video search result, we will only engage with provided videoId
        if 'regionCode' in locals():
          fullList = [[videoId, spark.sql("select cast(from_utc_timestamp(current_timestamp, 'America/Toronto') as string)").collect()[0][0], regionCode, 'mainpage']] 
        else:
          fullList = [[videoId, spark.sql("select cast(from_utc_timestamp(current_timestamp, 'America/Toronto') as string)").collect()[0][0],  'None', "mainpage"]] 

      videosDf = createSparkDf(fullList, schema = ["videoId", "fetchTimestampEst", "regionCode", "pageDetails"])

      #we fetch the video details of 25 videos at a time, the request can fetch up to 50 videos at a time but i limited it to 25 to avoid an unknown error. ##[{'message': 'The request specifies an invalid filter parameter.', 'domain': 'youtube.parameter', 'reason': 'invalidFilters', 'location': 'parameters.', 'locationType': 'other'}]##
      videoIdChunks = list(takeFewVideos(fullList, 25)) 
      for i in range(len(videoIdChunks)):
        videoIdChunk = videoIdChunks[i]
        videoList = []
        for j in videoIdChunk:
          videoList.append(j[0])

        videoListStr = str(videoList)[1:-1]
        videoListStr= videoListStr.replace("'", "")
        videoListStr = videoListStr.replace(" ", "")

        if i == 0:
          currentFetchVideoAttrDf = getVideoDetails(service, videoListStr)
        else:
          tempFetchVideoAttrDf = getVideoDetails(service, videoListStr)
          currentFetchVideoAttrDf = currentFetchVideoAttrDf.union(tempFetchVideoAttrDf)
          
      # we will join the initial df with fetched video ids using the search params and dataframe with video details for each of the fetched videoids. At the same time, publishedDate col is derived and if any duplicated records present, they are dropped. we are futher filtering the fetched videos by their title, description and tags to match the keywords passed by the user
      currentFetchVideoAttrDf = currentFetchVideoAttrDf.join(videosDf, currentFetchVideoAttrDf.videoId == videosDf.videoId, "fullouter").select(currentFetchVideoAttrDf["*"],to_date(col("publishedAt")).alias("publishedDate"), col("regionCode"),col("fetchTimestampEst")).filter(filterFurther(keyword)).dropDuplicates(['videoId'])
      
      #create the temp view to merge with the existing data, fetchTimestampEst will not be updated if the videoId is present in the new data
      currentFetchVideoAttrDf.createOrReplaceTempView("tempvideodetails")

      #update old common records
      spark.sql("""MERGE into delta.`{0}` as old USING tempvideodetails as new on new.videoId = old.videoId and (new.title <> old.title or new.description <> old.description or new.tags <> old.tags)  WHEN MATCHED THEN UPDATE SET  old.effectiveToDateEst = cast(from_utc_timestamp(current_timestamp, 'America/Toronto') as string), old.activeFlag = 'N'""".format(videosMountPath))

      ##insert common new records
      spark.sql("""insert into delta.`{0}` (videoId, kind, publishedAt, channelId, channelTitle, title, description, tags, duration, regionRestriction, viewCount, likeCount, favoriteCount, commentCount, publishedDate, regionCode, effectiveFromDateEst, effectiveToDateEst, activeFlag ) select new.videoId, new.kind, new.publishedAt, new.channelId, new.channelTitle, new.title, new.description, new.tags, new.duration, new.regionRestriction, new.viewCount, new.likeCount, new.favoriteCount, new.commentCount, new.publishedDate, new.regionCode, fetchTimestampEst as effectiveFromDateEst, '9999-12-31 99:99:99.999', 'Y' from tempvideodetails  as new INNER JOIN delta.`{0}` as old on new.videoId = old.videoId and (new.title <> old.title or new.description <> old.description or new.tags <> old.tags)""".format(videosMountPath))   

      ##insert new records
      spark.sql("""insert into delta.`{0}` (videoId, kind, publishedAt, channelId, channelTitle, title, description, tags, duration, regionRestriction, viewCount, likeCount, favoriteCount, commentCount, publishedDate, regionCode, effectiveFromDateEst, effectiveToDateEst, activeFlag ) select new.videoId, new.kind, new.publishedAt, new.channelId, new.channelTitle, new.title, new.description, new.tags, new.duration, new.regionRestriction, new.viewCount, new.likeCount, new.favoriteCount, new.commentCount, new.publishedDate, new.regionCode, fetchTimestampEst as effectiveFromDateEst, '9999-12-31 99:99:99.999', 'Y' from tempvideodetails  as new LEFT JOIN delta.`{0}` as old on new.videoId = old.videoId WHERE old.videoId IS NULL""".format(videosMountPath))


      youtubeVideoAttributesMain = spark.read.format("delta").load(videosMountPath)

      #we collect the videoIds whose comment counts are not 0. Need the videoId as python list
      CommentedVideoList = youtubeVideoAttributesMain.select(youtubeVideoAttributesMain.videoId).where(youtubeVideoAttributesMain.commentCount != '0').rdd.flatMap(lambda x: x).collect()

      for commentSearchTerm in commentSearchTermsList:
        for videoPos in range(len(CommentedVideoList)):
          videoId = CommentedVideoList[videoPos]
          if videoPos == 0:
            VideoComments = getComments(service, videoId, commentSearchTerm)
          else:
            nonFirstVideoComments = getComments(service, videoId, commentSearchTerm)
            VideoComments = VideoComments.union(nonFirstVideoComments)
        #create the temp view to merge with the existing data, commentFetchTimestampEst will not be updated if the videoId is present in the new data, commentPublishedDate is also created, duplicated commentIds are dropped
        VideoComments = VideoComments.select(VideoComments["*"],to_date(col("commentUpdatedAt")).alias("commentUpdatedDate")).dropDuplicates(['commentId']).dropna(subset="commentId").createOrReplaceTempView("tempcomments")
        
        
        spark.sql("""merge into delta.`{0}` as original using tempcomments as new on (original.commentId = new.commentId) when matched then update set commentParentId= new.commentParentId, commentKind= new.commentKind, commentEtag= new.commentEtag,  comment= new.comment, commentAuthor= new.commentAuthor, commentAuthorChannelId= new.commentAuthorChannelId, commentCanRate= new.commentCanRate, commentViewerRating= new.commentViewerRating, commentLikeCount= new.commentLikeCount, commentReplyCount= new.commentReplyCount, commentPublishedAt= new.commentPublishedAt, commentUpdatedAt= new.commentUpdatedAt, commentModerationStatus= new.commentModerationStatus, commentUpdatedDate = new.commentUpdatedDate,  commentsUpdateTimestampEst= new.commentsUpdateTimestampEst when not matched then insert (videoId, commentParentId, commentKind, commentEtag, commentId, comment, commentAuthor, commentAuthorChannelId, commentCanRate, commentViewerRating, commentLikeCount, commentReplyCount, commentPublishedAt, commentUpdatedAt, commentModerationStatus, commentUpdatedDate, commentsFetchTimestampEst, commentsUpdateTimestampEst) values (new.videoId, new.commentParentId, new.commentKind, new.commentEtag, new.commentId, new.comment, new.commentAuthor, new.commentAuthorChannelId, new.commentCanRate, new.commentViewerRating, new.commentLikeCount, new.commentReplyCount, new.commentPublishedAt, new.commentUpdatedAt, new.commentModerationStatus, new.commentUpdatedDate, new.commentsFetchTimestampEst, new.commentsUpdateTimestampEst) """. format(commentsMountPath)) 

        spark.sql('''optimize delta.`{0}`'''.format(commentsMountPath))
        spark.sql('''optimize delta.`{0}`'''.format(videosMountPath))
        spark.sql('''vacuum delta.`{0}` retain 0 hours'''.format(commentsMountPath))
        spark.sql('''vacuum delta.`{0}` retain 0 hours'''.format(videosMountPath))
    
    return VideoComments
  # ("successfully fetched comments")


  main(channel = channel, keyword = keyword, commentSearchTermsList = commentSearchTermsList, pathToServiceAccountJsonFile = pathToServiceAccountJsonFile, impersonationEmail = impersonationEmail, apiVersion = apiVersion, noResultsToFetch = noResultsToFetch, searchOrder = searchOrder, regionCode = regionCode, startDate = startDate, endDate = endDate)
  

In [0]:
a = fetchYoutubeComments(keyword = "air canada|aircanada|aeroplan", apiVersion = 'v3', noResultsToFetch = 5, commentSearchTermsList = ["air canada"], searchOrder = 'viewCount', regionCode = 'CA', startDate = '2024-10-20', endDate = '2024-10-25', impersonationEmail = 'sid-testing-001@avacado-315001.iam.gserviceaccount.com', pathToServiceAccountJsonFile = "/dbfs/mnt/Sandbox/pm_ap_conn/sid_test/avacado_315001_8a8bb7a78015.json")
display(a)